<a href="https://colab.research.google.com/github/bhuiyanmobasshir94/MultiCoNER/blob/main/notebooks/spark_nlp_multiconer_000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This is only to setup PySpark and Spark NLP on Colab
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2022-01-12 19:21:35--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/colab.sh [following]
--2022-01-12 19:21:35--  https://setup.johnsnowlabs.com/colab.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2022-01-12 19:21:36--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:44

In [9]:
!pip install gdown
!gdown --id 1AaaFcJ0Zzxr-fnlhlbp8mQePU6o8dFvk
!unzip -u "./BN_DATASET.zip" -d "./data"
!rm -f "./BN_DATASET.zip"
!cp -r ./data/BN-Bangla/. ./data && rm -rf ./data/BN-Bangla

Downloading...
From: https://drive.google.com/uc?id=1AaaFcJ0Zzxr-fnlhlbp8mQePU6o8dFvk
To: /content/BN_DATASET.zip
100% 1.38M/1.38M [00:00<00:00, 90.3MB/s]
Archive:  ./BN_DATASET.zip
   creating: ./data/BN-Bangla/
  inflating: ./data/BN-Bangla/bn_dev.conll  
  inflating: ./data/BN-Bangla/bn_train.conll  


In [10]:
import sparknlp
from sparknlp.annotator import *
from sparknlp.base import *

In [11]:
# if you have GPU
spark = sparknlp.start(gpu=True)
# spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  3.4.0
Apache Spark version:  3.0.3


In [12]:
with open("data/bn_train.conll") as f:
    c=f.read()

print (c[:200])

# id 5e80f5c9-1196-4906-826b-4cbdcaec7b6f	domain=train
২০১৮ _ _ O
এর _ _ O
সেরা _ _ O
(বর্ণানুক্রমিকভাবে _ _ O
তালিকাভুক্ত, _ _ O
র‍্যাঙ্ক _ _ O
করা _ _ O
হয়নি), _ _ O
এনপিআর _ _ B-CORP

# id ba55278


In [13]:
from sparknlp.training import CoNLL

training_data = CoNLL().readDataset(spark, "./data/bn_train.conll")
training_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|২০১৮ এর সেরা (বর্...|[[document, 0, 71...|[[document, 0, 71...|[[token, 0, 3, ২০...|[[pos, 0, 3, _, [...|[[named_entity, 0...|
|১৮২৫ সালে, তিনি ব...|[[document, 0, 81...|[[document, 0, 81...|[[token, 0, 3, ১৮...|[[pos, 0, 3, _, [...|[[named_entity, 0...|
|তিনি মার্কো মারুল...|[[document, 0, 53...|[[document, 0, 53...|[[token, 0, 3, তি...|[[pos, 0, 3, _, [...|[[named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



In [31]:
test_data = CoNLL().readDataset(spark, "./data/bn_dev.conll")
test_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|সমস্ত বেতন নিলামে...|[[document, 0, 10...|[[document, 0, 10...|[[token, 0, 4, সম...|[[pos, 0, 4, _, [...|[[named_entity, 0...|
|সংস্করণ ৩ প্রকাশে...|[[document, 0, 16...|[[document, 0, 16...|[[token, 0, 6, সং...|[[pos, 0, 6, _, [...|[[named_entity, 0...|
|একটি পাইলট (হাটন ...|[[document, 0, 11...|[[document, 0, 11...|[[token, 0, 3, এক...|[[pos, 0, 3, _, [...|[[named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



In [24]:
training_data.count()

15300

In [32]:
test_data.count()

800

In [26]:
# we use BERT Tiny
bert_annotator = BertEmbeddings.pretrained('small_bert_L2_128', 'en') \
 .setInputCols(["sentence",'token'])\
 .setOutputCol("bert")\
.setBatchSize(8)

small_bert_L2_128 download started this may take some time.
Approximate size to download 16.1 MB
[OK!]


In [33]:
test_data = bert_annotator.transform(test_data)
test_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|                bert|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|সমস্ত বেতন নিলামে...|[[document, 0, 10...|[[document, 0, 10...|[[token, 0, 4, সম...|[[pos, 0, 4, _, [...|[[named_entity, 0...|[[word_embeddings...|
|সংস্করণ ৩ প্রকাশে...|[[document, 0, 16...|[[document, 0, 16...|[[token, 0, 6, সং...|[[pos, 0, 6, _, [...|[[named_entity, 0...|[[word_embeddings...|
|একটি পাইলট (হাটন ...|[[document, 0, 11...|[[document, 0, 11...|[[token, 0, 3, এক...|[[pos, 0, 3, _, [...|[[named_entity, 0...|[[word_embeddings...|
+--------------------+--------------------+--------------------+--------------------+--------------------+

In [34]:
# let's transform and save our test dataset for evaluation
test_data.write.parquet("test_withEmbeds.parquet")

In [35]:
test_data.select("bert.result","bert.embeddings",'label.result').show()

+--------------------+--------------------+--------------------+
|              result|          embeddings|              result|
+--------------------+--------------------+--------------------+
|[সমস্ত, বেতন, নিল...|[[-1.1855662, 0.7...|[O, O, O, O, O, O...|
|[সংস্করণ, ৩, প্রক...|[[-1.3764335, 0.9...|[O, O, O, O, B-CW...|
|[একটি, পাইলট, (হা...|[[-0.863451, 0.20...|[O, O, O, O, O, B...|
|[প্রাইমাভেরা, ঝুঁ...|[[-1.2236071, 0.9...|[B-CW, O, O, O, O...|
|[১৮০০, -এর, দশকে,...|[[-1.1398509, 0.8...|[O, O, O, B-PER, ...|
|[চারজন, বিজয়ী, ড...|[[-1.0752249, 0.2...|[O, O, O, O, B-PE...|
|[এটি, ছিল, নাটকীয...|[[-0.777618, 0.51...|[O, O, O, O, O, B...|
|[এর, মধ্যে, একটি,...|[[-1.226814, 0.39...|[O, O, O, B-CORP,...|
|[এরলং, সুযোগ, সমা...|[[-0.9119873, 0.4...|[B-CW, O, O, O, O...|
|[যদিও, গির্জার, স...|[[-1.0261637, 0.6...|[O, O, O, O, B-PR...|
|[দ্য, ম্যাগনিফিসে...|[[-1.563148, 0.98...|[B-CW, I-CW, I-CW...|
|[দীর্ঘ, সময়, ধরে...|[[-1.3828295, 0.9...|[O, O, O, O, O, O...|
|[তা, সত্ত্বেও,, এ...|[[-

In [100]:
nerTagger = NerDLApproach()\
  .setInputCols(["sentence", "token", "bert"])\
  .setLabelColumn("label")\
  .setOutputCol("ner")\
  .setGraphFolder("/content/data")\
  .setMaxEpochs(5)\
  .setLr(0.001)\
  .setPo(0.005)\
  .setBatchSize(32)\
  .setEvaluationLogExtended(True) \
  .setEnableOutputLogs(True)\
  .setTestDataset("test_withEmbeds.parquet")

pipeline = Pipeline(
    stages = [
    bert_annotator,
    nerTagger
  ])

In [ ]:
ner_model = pipeline.fit(training_data)

In [92]:
# !git clone https://github.com/JohnSnowLabs/spark-nlp

In [96]:
# %cd spark-nlp/python/tensorflow
# %cd /content

In [94]:
# %env PYTHONPATH="/env/python:/content/spark-nlp/python/tensorflow/lib/ner"
# %env PYTHONPATH="$/env/python:/content/spark-nlp/python/tensorflow/lib/ner"
# %set_env PYTHONPATH=/content/spark-nlp/python/tensorflow/lib/ner:/env/python ## working solution
# %set_env PYTHONPATH=/env/python

In [90]:
# !source /content/ner_venv/bin/activate; python ner/create_models.py 13 128 220 /content/data

In [82]:
# !pip install virtualenv
# !virtualenv ner_venv
# !virtualenv -p /usr/bin/python3.6 ner_venv ## working solution

In [88]:
# !source /content/ner_venv/bin/activate; pip install tensorflow==1.15.0